In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Multi-fidelity (MF) optimization

In most cases it is better to do many cheap evaluations of an approximation to the target function than it is to only optimize the target function. This example demonstrates the 'multi-fidelity' capabilities of xopt. 

We follow the implementation of multi-fidelity bayesian optimization used in botorch https://botorch.org/tutorials/multi_fidelity_bo to optimize the synthetic test function AugmentedHartmann https://botorch.org/api/test_functions.html.

The difference between normal Bayesian optimization and MF optimization is that we specify a 'cost' to making observations at a given fidelity. For this example we assume a base cost of 5 and a fidelity cost between 0-1. The algorithm should make many observations at lower fidelity relative to higher fidelity, lowering the total observation cost. 

NOTE: The cost parameter is required to be the LAST element of the variables list. Also this method is best suited for parallel observations of the test function.

In [2]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann

import os
SMOKE_TEST = os.environ.get('SMOKE_TEST')

from botorch.acquisition.analytic import UpperConfidenceBound


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
# Make a proper input file. 
import yaml
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: multi_fidelity
  options:  
      n_initial_samples: 16
      n_steps: 6
      verbose: True
      generator_options:                     ## options for bayesian exploration acquisition function
          batch_size: 4                      ## batch size for parallelized optimization
          base_acq: custom_acq.acq           ## custom acquisition function if needed
          fixed_cost: 5.0                    ## fixed cost added onto fidelity cost
          use_gpu: False


simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
if 1:
    config['algorithm']['options']['n_steps'] = 3
    config['algorithm']['options']['generator_options']['batch_size'] = 1
    config['algorithm']['options']['generator_options']['num_restarts'] = 2
    config['algorithm']['options']['generator_options']['raw_samples'] = 2
    config['algorithm']['options']['generator_options']['base_acq'] = None

X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3+107.g57ab691.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: multi_fidelity
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
  options:
    n_initial_samples: 16
    n_steps: 3
    verbose: true
    generator_options: {batch_size: 1, base_acq: null, fixed_cost: 5.0, use_gpu: false,
      num_restarts: 2, raw_samples: 2}
    custom_model: !!python/name:xopt.bayesian.models.models.create_multi_fidelity_model ''
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

#executor = PoolExecutor()
# This will also work. 
executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-09-02T11:00:00-05:00
started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x000001F0EF9EFA90>
submitting initial candidates at time 2021-09-02T11:00:00-05:00
starting optimization loop
Model creation time: 0.12 s
Candidate generation time: 5.394 s
Candidate(s): tensor([[0.4747, 0.4298, 0.8082, 0.2263, 0.3694, 0.7291, 0.0079]],
       dtype=torch.float64)
submitting candidates at time 2021-09-02T11:00:05-05:00
Model creation time: 0.1544 s
Candidate generation time: 5.055 s
Candidate(s): tensor([[0.5472, 0.4419, 0.8152, 0.2499, 0.3145, 0.8486, 0.0068]],
       dtype=torch.float64)
submitting candidates at time 2021-09-02T11:00:11-05:00
Model creation time: 0.2461 s
Candidate generation time: 4.994 s
Candidate(s): tensor([[0.4736, 0.4338, 0.8396, 0.3410, 0.2744, 0.7720, 0.0300]],
       dtype=torch.float64)
submitting candidates at time 2021-09-02T11:00:16-05:00


### Get highest fidelity global optimum

In [7]:
# create generator object
gen = MultiFidelityGenerator(X.vocs)

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [9]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237

tensor([-1.5846], dtype=torch.float64)